In [1]:
from pytube import YouTube
from pytube import Channel
from pytube import extract
import pandas as pd
from collections import defaultdict
import os

In [2]:
file_test = 'video_urls_FULL.csv'
df = pd.read_csv(file_test)
df = df.drop(columns='Unnamed: 0')

In [3]:
df.shape

(46532, 2)

In [4]:
df = df[255:265].reset_index(drop= True)

In [5]:
df

,channel,url
0,UPA_Federal,youtube.com/watch?v=b4vPOoDtttE
1,UPA_Federal,youtube.com/watch?v=OYS2pt9DVsY
2,UPA_Federal,youtube.com/watch?v=yq5LTOSht7M
3,UPA_Federal,youtube.com/watch?v=5Q9yXVlAtXA
4,UPA_Federal,youtube.com/watch?v=f0QNt-OsmgE
5,UPA_Federal,youtube.com/watch?v=Tb_pMJIQOTk
6,CSIFNacional,youtube.com/watch?v=uTiPr5n1daY
7,CSIFNacional,youtube.com/watch?v=-vehbrcFsFQ
8,CSIFNacional,youtube.com/watch?v=RjSTUo3UM7M
9,CSIFNacional,youtube.com/watch?v=SwN9yBezHKg


In [6]:
def video_metadata_v2(df):
    keys = ['channel', 'url', 'id', 'title', 'author', 'description', 'keywords', 'length_in_secs', 
            'nbr_views', 'age_resticted', 'publication_date', 'rating']

    metadata = defaultdict(list)

    for idx in range(0,df.shape[0]):

        # Access the video url from data frame
        video_url = df['url'].iloc[idx]
        #print(f'Video_url: {video_url}')
        channel_name = df['channel'].iloc[idx]
        #print(f'Channel: {channel_name}')

        # Create a youtube object to use the API
        yt = YouTube(video_url)
        
        # Extract metadata
        info = [channel_name, video_url, extract.video_id(video_url), yt.title, yt.author, yt.description,
        yt.keywords, yt.length, yt.views, yt.age_restricted, yt.publish_date, yt.rating]

        # Store in dictionary 
        for k, v in zip(keys,info):
            metadata[k].append(v)
                    
        # Track 
        if idx % 1000 == 0:
            print(f'We are in video {idx} in channel {yt.author}')

    return metadata


In [7]:
# Getting all the metadata 
info = video_metadata_v2(df)

We are in video 0 in channel UPA - Unión de Pequeños Agricultores y Ganaderos


In [8]:
video_metadata = pd.DataFrame.from_dict(info)
video_metadata

,channel,url,id,title,author,description,keywords,length_in_secs,nbr_views,age_resticted,publication_date,rating
0,UPA_Federal,youtube.com/watch?v=b4vPOoDtttE,b4vPOoDtttE,El sector se une en defensa del aceite de oliva,UPA - Unión de Pequeños Agricultores y Ganaderos,Acciones▾Compartir esto▾Más nuevosMás antiguos...,"[Lorenzo Ramos, UPA, aceite de oliva, almacena...",270,46,False,2011-03-15,None
1,UPA_Federal,youtube.com/watch?v=OYS2pt9DVsY,OYS2pt9DVsY,Reunión MARM-OPAs medidas de apoyo a la ganadería,UPA - Unión de Pequeños Agricultores y Ganaderos,Lorenzo Ramos ha asistido a la reunión con la ...,"[Lorenzo Ramos, Rosa Aguilar, ganadería, UPA]",298,25,False,2011-03-08,None
2,UPA_Federal,youtube.com/watch?v=yq5LTOSht7M,yq5LTOSht7M,"FADEMUR - 8 de marzo, día de la mujer trabajad...",UPA - Unión de Pequeños Agricultores y Ganaderos,,"[FADEMUR, mujer rural, mujeres]",293,128,False,2011-03-08,None
3,UPA_Federal,youtube.com/watch?v=5Q9yXVlAtXA,5Q9yXVlAtXA,Reunión en el MARM para analizar la crisis gan...,UPA - Unión de Pequeños Agricultores y Ganaderos,"Lorenzo Ramos y Román Santalla, secretario gen...","[Lorenzo Ramos, MARM, ganadería, ganaderos, cr...",288,48,False,2011-03-04,None
4,UPA_Federal,youtube.com/watch?v=f0QNt-OsmgE,f0QNt-OsmgE,Fademur Catalunya 2011,UPA - Unión de Pequeños Agricultores y Ganaderos,La Federació d'Associacions de Dones Rurals (F...,"[FADEMUR, Catalunya, dones rurals, pageses]",366,61,False,2011-02-17,None
5,UPA_Federal,youtube.com/watch?v=Tb_pMJIQOTk,Tb_pMJIQOTk,Dacian Ciolos saluda a la Unión de Pequeños Ag...,UPA - Unión de Pequeños Agricultores y Ganaderos,El Comisario de Agricultura y Desarrollo Rural...,"[UPA, agricultura, ganadería, PAC, Dacian Ciolos]",506,286,False,2010-08-02,None
6,CSIFNacional,youtube.com/watch?v=uTiPr5n1daY,uTiPr5n1daY,El sistema educativo público está desbordado p...,CSIF Nacional,,"[Sindicato, CSI-F]",27,53,False,2022-12-20,None
7,CSIFNacional,youtube.com/watch?v=-vehbrcFsFQ,-vehbrcFsFQ,CSIF exige más medios para un Sanidad en declive,CSIF Nacional,,"[Sindicato, CSI-F]",106,94,False,2022-12-19,None
8,CSIFNacional,youtube.com/watch?v=RjSTUo3UM7M,RjSTUo3UM7M,CSIF defiende el acceso a la nueva promoción i...,CSIF Nacional,,"[Sindicato, CSI-F]",7,56,False,2022-12-18,None
9,CSIFNacional,youtube.com/watch?v=SwN9yBezHKg,SwN9yBezHKg,CSIF exige más medios para un Sanidad en declive,CSIF Nacional,,"[Sindicato, CSI-F]",7,24,False,2022-12-18,None


In [12]:
type(video_metadata.length_in_secs.iloc[0])

numpy.int64

In [8]:
#file_test = '../Data_collection/data/videos_metadata.csv'
df = pd.read_csv(file_test)

In [9]:
df

,channel,url,id,title,author,description,keywords,length_in_secs,nbr_views,age_resticted,publication_date,rating
0,UPA_Federal,youtube.com/watch?v=b4vPOoDtttE,b4vPOoDtttE,El sector se une en defensa del aceite de oliva,UPA - Unión de Pequeños Agricultores y Ganaderos,Acciones▾Compartir esto▾Más nuevosMás antiguos...,"['Lorenzo Ramos', 'UPA', 'aceite de oliva', 'a...",270,46,False,2011-03-15,NaN
1,UPA_Federal,youtube.com/watch?v=OYS2pt9DVsY,OYS2pt9DVsY,Reunión MARM-OPAs medidas de apoyo a la ganadería,UPA - Unión de Pequeños Agricultores y Ganaderos,Lorenzo Ramos ha asistido a la reunión con la ...,"['Lorenzo Ramos', 'Rosa Aguilar', 'ganadería',...",298,25,False,2011-03-08,NaN
2,UPA_Federal,youtube.com/watch?v=yq5LTOSht7M,yq5LTOSht7M,"FADEMUR - 8 de marzo, día de la mujer trabajad...",UPA - Unión de Pequeños Agricultores y Ganaderos,NaN,"['FADEMUR', 'mujer rural', 'mujeres']",293,128,False,2011-03-08,NaN
3,UPA_Federal,youtube.com/watch?v=5Q9yXVlAtXA,5Q9yXVlAtXA,Reunión en el MARM para analizar la crisis gan...,UPA - Unión de Pequeños Agricultores y Ganaderos,"Lorenzo Ramos y Román Santalla, secretario gen...","['Lorenzo Ramos', 'MARM', 'ganadería', 'ganade...",288,48,False,2011-03-04,NaN
4,UPA_Federal,youtube.com/watch?v=f0QNt-OsmgE,f0QNt-OsmgE,Fademur Catalunya 2011,UPA - Unión de Pequeños Agricultores y Ganaderos,La Federació d'Associacions de Dones Rurals (F...,"['FADEMUR', 'Catalunya', 'dones rurals', 'page...",366,61,False,2011-02-17,NaN
5,UPA_Federal,youtube.com/watch?v=Tb_pMJIQOTk,Tb_pMJIQOTk,Dacian Ciolos saluda a la Unión de Pequeños Ag...,UPA - Unión de Pequeños Agricultores y Ganaderos,El Comisario de Agricultura y Desarrollo Rural...,"['UPA', 'agricultura', 'ganadería', 'PAC', 'Da...",506,286,False,2010-08-02,NaN
6,CSIFNacional,youtube.com/watch?v=uTiPr5n1daY,uTiPr5n1daY,El sistema educativo público está desbordado p...,CSIF Nacional,NaN,"['Sindicato', 'CSI-F']",27,53,False,2022-12-20,NaN
7,CSIFNacional,youtube.com/watch?v=-vehbrcFsFQ,-vehbrcFsFQ,CSIF exige más medios para un Sanidad en declive,CSIF Nacional,NaN,"['Sindicato', 'CSI-F']",106,94,False,2022-12-19,NaN
8,CSIFNacional,youtube.com/watch?v=RjSTUo3UM7M,RjSTUo3UM7M,CSIF defiende el acceso a la nueva promoción i...,CSIF Nacional,NaN,"['Sindicato', 'CSI-F']",7,56,False,2022-12-18,NaN
9,CSIFNacional,youtube.com/watch?v=SwN9yBezHKg,SwN9yBezHKg,CSIF exige más medios para un Sanidad en declive,CSIF Nacional,NaN,"['Sindicato', 'CSI-F']",7,24,False,2022-12-18,NaN
